In [1]:
from extract_ner import get_all_ners,get_all_projects
from pdf_analyzer.config_load import load_config
from pdf_analyzer.api import API
from omegaconf import OmegaConf

In [2]:
from transformers import pipeline

pipe = pipeline("token-classification", model="dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
server_config = load_config("config/api/grobid-server-config.yaml")
extract_config = load_config("config/api/api-base-config.yaml")
print("SERVER_CONFIG\n"+OmegaConf.to_yaml(server_config))
print("CLOUD_CONFIG\n"+OmegaConf.to_yaml(extract_config))

base_api = API.BaseAPI(extract_config,server_config)

files = base_api.proccesed_files

SERVER_CONFIG
url:
  protocol: http
  api_domain: yordi111nas.synology.me
  port: 8070

CLOUD_CONFIG
data:
  data_dir: data/PDFs
  format: .pdf
  recursive: true
grobid:
  cache: true
  cache_dir: data/xmls
  operation_key: processFulltextDocument
  format: .grobid.tei.xml
  recursive: true

GROBID server is up and running
grobid_logger: 2024-05-22 13:19:01,128 | INFO | API.py:44 | 27256 >>> All files have been process by the api


In [4]:
all_orgs_rel,all_orgs=get_all_ners(files,pipe)
all_orgs_rel


[{'name': 'PolarB',
  'score': 0.8971113,
  'type': 'ORG',
  'org_id': 0,
  'paper_id': 10},
 {'name': 'Centre National de la Recherche Scientifique of',
  'score': 0.902651,
  'type': 'ORG',
  'org_id': 1,
  'paper_id': 10},
 {'name': 'CNRS',
  'score': 0.9043675,
  'type': 'ORG',
  'org_id': 2,
  'paper_id': 10},
 {'name': 'Pérez & Granger',
  'score': 0.8425125,
  'type': 'ORG',
  'org_id': 3,
  'paper_id': 13},
 {'name': 'NASA',
  'score': 0.9977794,
  'type': 'ORG',
  'org_id': 4,
  'paper_id': 16},
 {'name': 'NAS',
  'score': 0.81430924,
  'type': 'ORG',
  'org_id': 5,
  'paper_id': 16},
 {'name': 'SRG',
  'score': 0.86928785,
  'type': 'ORG',
  'org_id': 6,
  'paper_id': 18},
 {'name': 'Russian Space Agency',
  'score': 0.9969481,
  'type': 'ORG',
  'org_id': 7,
  'paper_id': 18},
 {'name': 'Ros kosmos',
  'score': 0.97922397,
  'type': 'ORG',
  'org_id': 8,
  'paper_id': 18},
 {'name': 'Russian Academy of Sciences',
  'score': 0.9965502,
  'type': 'ORG',
  'org_id': 9,
  'paper

In [5]:
all_orgs

[{'name': 'PolarB', 'type': 'ORG', 'org_id': 0},
 {'name': 'Centre National de la Recherche Scientifique of',
  'type': 'ORG',
  'org_id': 1},
 {'name': 'CNRS', 'type': 'ORG', 'org_id': 2},
 {'name': 'Pérez & Granger', 'type': 'ORG', 'org_id': 3},
 {'name': 'NASA', 'type': 'ORG', 'org_id': 4},
 {'name': 'NAS', 'type': 'ORG', 'org_id': 5},
 {'name': 'SRG', 'type': 'ORG', 'org_id': 6},
 {'name': 'Russian Space Agency', 'type': 'ORG', 'org_id': 7},
 {'name': 'Ros kosmos', 'type': 'ORG', 'org_id': 8},
 {'name': 'Russian Academy of Sciences', 'type': 'ORG', 'org_id': 9},
 {'name': 'Space Research Institute', 'type': 'ORG', 'org_id': 10},
 {'name': 'IKI', 'type': 'ORG', 'org_id': 11},
 {'name': 'Deutsches Zentrum für Luft-und Raumfahrt',
  'type': 'ORG',
  'org_id': 12},
 {'name': 'DLR', 'type': 'ORG', 'org_id': 13},
 {'name': 'SR', 'type': 'ORG', 'org_id': 14},
 {'name': 'Lavochkin Association', 'type': 'ORG', 'org_id': 15},
 {'name': 'NPOL', 'type': 'ORG', 'org_id': 16},
 {'name': 'Max Pla

In [6]:
all_projects,all_projects_relation=get_all_projects(files)
all_projects

[{'project_name': 'NASA',
  'project_federal_id': '80NSSC23K1115',
  'project_id': 0},
 {'project_name': 'MURI',
  'project_federal_id': 'N00014-18-1-2670',
  'project_id': 1},
 {'project_name': 'ONR',
  'project_federal_id': '#N00014-13-1-0023',
  'project_id': 2}]

In [7]:
all_projects_relation

[{'project_name': 'NASA',
  'project_federal_id': '80NSSC23K1115',
  'project_id': 0,
  'paper_id': 16},
 {'project_name': 'NASA',
  'project_federal_id': '80NSSC23K0911',
  'project_id': 0,
  'paper_id': 16},
 {'project_name': 'MURI',
  'project_federal_id': 'N00014-18-1-2670',
  'project_id': 1,
  'paper_id': 22},
 {'project_name': 'ONR',
  'project_federal_id': '#N00014-13-1-0023',
  'project_id': 2,
  'paper_id': 24},
 {'project_name': 'ONR',
  'project_federal_id': '#W911NF-13-1-0246',
  'project_id': 2,
  'paper_id': 24}]

In [8]:
import re
def extract_award_identifiers(text):
    # Expresiones regulares para identificadores de premios
    regex_patterns = {
        "NIH": r'(?:#)?\b[1-9][A-Z\d]{3}[A-Z]{2}\d{6}(?:-[AS]?\d+)?\b',
        "DOD": r'(?:#)?\b[A-Z\d]{6}-\d{2}-[123]-\d{4}\b',
        "NASA": r'(?:#)?\b(?:80|NN)[A-Z]+\d{2}[A-Z\d]+\b',
        "Education": r'(?:#)?\b[A-Z]+\d+[A-Z]\d{2}[A-Z\d]+\b'
    }

    # Lista para almacenar todos los identificadores únicos encontrados
    all_identifiers = []

    # Aplicar cada expresión regular al texto y almacenar los resultados
    for pattern in regex_patterns.values():
        identifiers = re.findall(pattern, text)
        all_identifiers.extend(identifiers)

    # Eliminar duplicados
    unique_identifiers = list(set(all_identifiers))

    return unique_identifiers

In [9]:


texto_ejemplo = """
Aquí hay algunos identificadores de premios:

NIH: 1R01CA123456-01A1, #2R01CA654321-03B2
DOD: W81XWH-22-1-1234, #W912D1-21-3-9012
NASA: NNC21AA01H, #GRC20G-1234, NNC21AA02A
Department of Education: P031B201234, #T042A190567
"""

# Extraer identificadores de premios del texto de ejemplo
identificadores = extract_award_identifiers(texto_ejemplo)

# Imprimir los identificadores encontrados para cada agencia
for ids in identificadores:
    print(f"Identificadores de premios {ids}")

Identificadores de premios P031B201234
Identificadores de premios #2R01CA654321
Identificadores de premios 1R01CA123456
Identificadores de premios #W912D1-21-3-9012
Identificadores de premios NNC21AA01H
Identificadores de premios W81XWH-22-1-1234
Identificadores de premios #T042A190567
Identificadores de premios NNC21AA02A


In [10]:
def combine_identifiers_with_names(text):
    # Obtener identificadores de premios del texto
    identifiers = extract_award_identifiers(text)
    
    # Lista para almacenar nombres y códigos asociados
    names_and_codes = []

    # Iterar sobre los identificadores y buscar los nombres correspondientes
    for identifier in identifiers:
        # Buscar el nombre del proyecto antes del identificador
        match = re.search(rf'(\w+)\s+(?:grant|award|code)\s+{re.escape(identifier)}', text, re.IGNORECASE)
        if match:
            names_and_codes.append((match.group(1), identifier))

    return names_and_codes

In [11]:
def get_projects_names(text):
    # Obtener identificadores de premios del texto
    identifiers = extract_award_identifiers(text)
    
    # Lista para almacenar nombres y códigos asociados
    names_and_codes = []

    # Iterar sobre los identificadores y buscar los nombres correspondientes
    for identifier in identifiers:
        # Buscar el nombre del proyecto antes del identificador
        match = re.search(rf'(\w+)\s+(?:grant|award|code)\s+{re.escape(identifier)}', text, re.IGNORECASE)
        if match:
            names_and_codes.append(match.group(1))

    return names_and_codes

In [12]:
texto_ejemplo = """
Aquí hay algunos identificadores de premios con nombres asociados:

MURI grant N00014-18-1-2670, MURI award N00014-18-1-2670
DOD Award W81XWH-18-1-1234, Cooperative Agreement W911NF-20-2-5678
NASA Grant NNC21AA01H, Research Cooperative Agreement GRC20G-1234, NNC21AA02A
Department of Education Grant P031B201234, Grant T042A190567
"""

# Extraer nombres asociados con códigos del texto de ejemplo
nombres_con_codigos = get_projects_names(texto_ejemplo)
nombres_con_codigos

['DOD', 'Education', 'NASA', 'MURI']

In [13]:
nombres_con_codigos

['DOD', 'Education', 'NASA', 'MURI']